In [3]:
import nltk, re, pprint
import numpy as np

from collections import Counter
from nltk import word_tokenize
from urllib import request

In [49]:
def ie_preprocess(document):
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [94]:
filename = "/Users/williamshyr/Documents/MedHacks/cohere_proj/timecube_raw.txt"
f = open(filename, 'r')
raw = f.read()

In [95]:
sentences = ie_preprocess(raw)

In [103]:
sentences[3]

[('The', 'DT'),
 ('bible', 'JJ'),
 ('time', 'NN'),
 ('was', 'VBD'),
 ('wrong', 'JJ'),
 ('then', 'RB'),
 ('and', 'CC'),
 ('it', 'PRP'),
 ('proved', 'VBD'),
 ('wrong', 'JJ'),
 ('today', 'NN'),
 ('.', '.')]

In [111]:
# grammar = "NP: {<DT|PP\$>?<JJ>*<NN>}"
grammar = r"""
    NBAR:
        {<DT|PP\$>?<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
    NP:
        {<NBAR>}
        {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
"""
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentences[3])
print(result)
# print(len(result))
# maxLen = 0
# for i in range(len(sentences)):
#     result = cp.parse(sentences[i])
#     for i in range(len(result)):
#         if isinstance(result[i], nltk.tree.Tree):
#             if len(result[i]) > maxLen:
#                 maxLen = len(result[i])

(S
  (NP (NBAR The/DT bible/JJ time/NN))
  was/VBD
  wrong/JJ
  then/RB
  and/CC
  it/PRP
  proved/VBD
  (NP (NBAR wrong/JJ today/NN))
  ./.)


In [119]:
grammar = "NP: {<DT|PP\$>?<JJ>*<NN>}"
# grammar = r"""
#     NBAR:
#         {<DT|PP\$>?<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
        
#     NP:
#         {<NBAR>}
#         {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...
# """
cp = nltk.RegexpParser(grammar)
# result = cp.parse(sentences[3])
# print(result)
# print(len(result))
maxLen = 0
for i in range(len(sentences)):
    result = cp.parse(sentences[i])
    for i in range(len(result)):
        if isinstance(result[i], nltk.tree.Tree):
            if len(result[i]) > maxLen:
                maxLen = len(result[i])
print(maxLen)

5


In [109]:
grammar = "NP: {<DT|PP\$>?<JJ>*<NN>}"
cp = nltk.RegexpParser(grammar)
result = cp.parse(sentences[3])
print(result)
# print(len(result))
# maxLen = 0
# for i in range(len(sentences)):
#     result = cp.parse(sentences[i])
#     for i in range(len(result)):
#         if isinstance(result[i], nltk.tree.Tree):
#             if len(result[i]) > maxLen:
#                 maxLen = len(result[i])

(S
  (NP The/DT bible/JJ time/NN)
  was/VBD
  wrong/JJ
  then/RB
  and/CC
  it/PRP
  proved/VBD
  (NP wrong/JJ today/NN)
  ./.)


In [123]:
# get training and testing data
from nltk.corpus import conll2000
test_sents = conll2000.chunked_sents('test.txt', chunk_types=['NP'])
train_sents = conll2000.chunked_sents('train.txt', chunk_types=['NP'])

In [125]:
NPChunker = ChunkParser(train_sents)

In [124]:
class ChunkParser(nltk.ChunkParserI):
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
            for sent in train_sents]
        self.tagger = nltk.TrigramTagger(train_data)
    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
            in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [126]:
print(NPChunker.evaluate(test_sents))

ChunkParse score:
    IOB Accuracy:  93.3%
    Precision:     82.5%
    Recall:        86.8%
    F-Measure:     84.6%


In [127]:
ChunkParser.parse(sentences[3])

TypeError: parse() missing 1 required positional argument: 'sentence'

NotImplementedError: Use label() to access a node label.